# Research: Music of Big Cities

Question: how is the music played on the way to work on Monday morning different from the one played on Wednesday or
at the end of the work week?

Based on the data for Moscow and St.Petersburg, compare what and in what mode their residents listen.

## Imports

In [1]:
import pandas as pd

Read file *music_project.csv*

In [2]:
df = pd.read_csv(r'/datasets/music_project.csv') # <reading data file with saving to df>

Getting the first 10 rows of the table

In [3]:
df.head(10) 

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


General information about the data




In [4]:
df.info

<bound method DataFrame.info of          userID                              Track            artist  \
0      FFB692EC                  Kamigata To Boots  The Mass Missile   
1      55204538        Delayed Because of Accident  Andreas Rönnberg   
2        20EC38                  Funiculì funiculà       Mario Lanza   
3      A3DD03C9              Dragons in the Sunset        Fire + Ice   
4      E2DC1FAE                        Soul People        Space Echo   
...         ...                                ...               ...   
65074  729CBB09                            My Name            McLean   
65075  D08D4A55  Maybe One Day (feat. Black Spade)       Blu & Exile   
65076  C5E3A0D5                          Jalopiina               NaN   
65077  321D0506                      Freight Train     Chas McDevitt   
65078  3A64EF84          Tell Me Sweet Little Lies      Monica Lopez   

            genre            City        time        Day  
0            rock  Saint-Petersburg  20:28:3

Let's consider the information received.

There are 7 colomns in total:

* userID;
* Track (name of the track);  
* artist (artist name);
* genre (genre of the track);
* City (cityt where the audition took place);
* time (the time at which user listened to the track);
* Day (day of week).

    The number of values in the columns varies. This indicates that there is a missing value in the data.



**Conclusions**

Each row of the table contains information about compositions of a certain genre in a certain performance that users listened to in one of the cities at a certain time and day of the week. Two problems to deal with are gaps and bad column names. The 'time', 'day', and 'City' columns are especially valuable for testing working hypotheses. Genre column data will reveal the most popular genres.

# Data preprocessing

Let's eliminate gaps, rename columns, and check the data for duplicates.

Get a list of column names.

In [5]:
df.columns # list of column names. Let's check them.

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Column names contain spaces that can make data difficult to access and process.

Let's rename the columns for the convenience of further work.



In [6]:
new_names = ['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'weekday']
df.set_axis(new_names, axis='columns', inplace=True)

In [7]:
df.columns # Check the results

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

Let's check the data for gaps by calling a set of methods for summing missing values.

In [8]:
df.isnull().sum() # total number of omission identified by the method isnull()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Empty values indicate that not all information is available for some tracks.
The reasons may vary. Each individual case must by analyzed and the cause identified.

Replace the missing values in the track and artist columns with the string "unknown". After this operation, check the table no longer contains gaps.

In [9]:
# Replace missing values in 'track_name' column with 'unknown' string.
df['track_name'] = df['track_name'].fillna('unknown')



In [10]:
# Replace missing values in 'artist_name' column with 'unknown' string.
df['artist_name'] = df['artist_name'].fillna('unknown')

In [11]:
# Check: calculating the total number of missing values.
df.isnull().sum()

user_id           0
track_name        0
artist_name       0
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Delete empty values in the column with genres and after that check that there are no more of them.

In [15]:
# Delete missing values in the 'genre_name' column.
df.dropna(subset=['genre_name'], inplace=True)

In [16]:
df.isnull().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

Establish the presence of duplicates. If there any -> delete and check if they all deleted.

In [17]:
# Getting the total number of duplicates.
df.duplicated().sum() 

3755

In [18]:
# Removing all duplicates from the table.
df = df.drop_duplicates().reset_index(drop=True)

In [19]:
# Check for duplicates.
df.duplicated().sum()

0

Duplicates may have appeared due to a failure in data recording. It is worth paying attention and sorting out the reasons for the appearance of such "information garbage".

Let's create:

1) "genre_list" - list of unique values for the genre.

2) Function *find_genre()* to find implicit duplicates in a column with genres (for example, when the name of the same genre is written in different words). 


In [20]:
# Saving in the variable genres_list list of unique values.
genres_list = df['genre_name'].unique()

In [21]:
# Function "find_genre()"

def find_genre(genre):
    """
       Takes as a parameter a string with the name of the desired genre,
       a counter variable is decalred in the body,
       then the for loop goes through the list of unique values,
       if the next item in the list is equal to the function parameter
       then the counter value is increased by 1.
       The function returns the counter value.
    """
    count = 0
    for g in genres_list:
        if g == genre:
            count += 1
    return count


Let's call the *find_genre()* to find different hip-hop genre names in a table.

The correct name is *hiphop*. 
Let's look for other options:

* hip
* hop
* hip-hop


In [22]:
find_genre('hip')

1

In [23]:
find_genre('hop')

0

In [24]:
find_genre('hip-hop')

0

Let's declare the 'find_hip_hop()' function, which replaces the incorrect name of the genre in the 'genre_name' column with 'hiphop' and checks if the replacement was successful.
So correct all the spellings that the check revealed.

In [25]:
# Function 'find_hip_hop'

def find_hip_hop(df, wrong):
    """
    Takes as parameters the table and the wrong name,
    a special method is applied to the 'genre_name' column,
    which replaces the second parameter with the string 'hiphop',
    the function returns the number of column values calculated by count()
    """
    df['genre_name'] = df['genre_name'].replace(wrong, 'hiphop')
    result = df[df['genre_name'] == wrong]['genre_name'].count()
    return result

In [26]:
find_hip_hop(df, 'hip')

0

Get general information about the data. Check the cleaning was successful.

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
user_id        60126 non-null object
track_name     60126 non-null object
artist_name    60126 non-null object
genre_name     60126 non-null object
city           60126 non-null object
time           60126 non-null object
weekday        60126 non-null object
dtypes: object(7)
memory usage: 3.2+ MB


**Conclusions**

At the preprocesiing stage, not only gaps and problems with column names were found in the data, but also all kinds of duplicates. Removing them will make the analysis more accurate. Since it is important to save genre information for analysis, I do not just remove all missing values, but fill in the missing artist names and track titles. Column names are now correct and convenient for further work.

# Is it really people in different cities listens different music.

It was hypothesized that in Moscow and St.Petersburg, users listen to music in different ways. Let's check this assumption using data on three days of the week - Monday, Wednesday and Friday.

For each city, let's set the number of songs with a known genre listened to these days, and compare the results.

Let's group the data by city and call the 'count()' method to count the compositions for which the genre is known.

In [28]:
df.groupby('city')['genre_name'].count()

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

There are more auditions in Moscow than in St.Petersburg, but this does not mean that Moscow is more active. Yandex.Music has more users in Moscow, so the numbers are comparable.

Let's goup the data by the day of the week and calculate the songs listened on Monday, Wednesday and Friday for which the genre is known.

In [29]:
df.groupby('weekday')['genre_name'].count()

weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

**Conclusions:**<br> 
Monday and Friday are music time; on Wednesday, users are a little more involved in work.

Let's create the function 'number_tracks()', which takes as parameters a table, the day of the week and the name of the city, and returns the number of listened songs for which the genre is known. 
Check the number of listened songs for each city and a weekday.

In [30]:
# Function number_tracks()

def number_tracks(df, day, city):
    """
      The function with three parameters: df, day, city
      the track_list variable stores those rows of the df table for which
      the value in the 'weekday' column is equal to the day parameter
      and at the same time the value in the 'city' column is equal to
      the city parameter.
    return track_list_count
    """
    track_list = df[(df['weekday']==day)&(df['city']==city)]
    track_list_count = track_list['genre_name'].count()
    return track_list_count

In [36]:
# number of tracks for Moscow on Monday.
number_tracks(df, 'Monday', 'Moscow') 

15347

In [37]:
# number of tracks for St. Petersburg on Monday.
number_tracks(df, 'Monday', 'Saint-Petersburg')

5519

In [38]:
# number of tracks for Moscow on Wednesday.
number_tracks(df, 'Wednesday', 'Moscow')

10865

In [39]:
# number of tracks for St. Petersburg on Wednesday.
number_tracks(df, 'Wednesday', 'Saint-Petersburg')

6913

In [40]:
# number of tracks for Moscow on Friday
number_tracks(df, 'Friday', 'Moscow') 

15680

In [41]:
# number of tracks for St. Petersburg on Friday
number_tracks(df, 'Friday', 'Saint-Petersburg')

5802

Let's summarize the received information in one table, where ['city', 'monday', 'wednesday', 'friday'] are the names of the columns.


In [42]:
# table with received data
data = [['Moscow', 15347, 10865, 15680], ['Saint-Petersburg', 5519, 6913, 5802]]
columns = ['city', 'monday', 'wednesday', 'friday']
table = pd.DataFrame(data=data, columns=columns)
table

,city,monday,wednesday,friday
0,Moscow,15347,10865,15680
1,Saint-Petersburg,5519,6913,5802


**Conslusions**

The results show that, relative to Wednesday, music in St. Petersburg and Moscow is listened to in a 'mirror' manner: in Moscow peaks fall on Monday and Friday, and on Wednesday listening time decreases. Where in St. Petersburg Wednesday is the of the greatest interest in music, and on Monday and Friday it is less, and almost equally less.

# Monday morning and Friday evening - different music or the same?

Looking for an answer to the question, which genres prevail in different cities on Monday morning and Friday evening. There is speculation that on Monday morning, users listen to more uplifting music (such as pop), and more dance music (such as electronics) on Friday nights.

Let's get data tables for Moscow *moscow_general* and for St. Petersburg *spb_general*.

In [43]:
moscow_general = df[df['city']=='Moscow']

In [44]:
spb_general = df[df['city']=='Saint-Petersburg']

Let's create function *genre_weekday()* that returns a list of genres for the requested day of the week and time of day from such and such hour to such and such.

In [45]:

def genre_weekday(df, day, time1, time2):
    """
    
    """
    genre_list = df[(df['weekday']==day)&(df['time'] > time1)&(df['time'] < time2)]
    genre_list_sorted = genre_list.groupby('genre_name')['genre_name'].count().sort_values(ascending=False).head(10)
    return genre_list_sorted

Let's compare the results obtained in the table for Moscov and St. Petersburg on Morning morning (from 7 to 11) and on Friday evening (from 17 to 23).

In [47]:
genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00')

genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: genre_name, dtype: int64

In [48]:
genre_weekday(spb_general, 'Monday', '07:00:00', '11:00:00')

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre_name, dtype: int64

In [49]:
genre_weekday(moscow_general, 'Friday', '17:00:00', '23:00:00')

genre_name
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre_name, dtype: int64

In [50]:
genre_weekday(spb_general, 'Friday', '17:00:00', '23:00:00')

genre_name
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre_name, dtype: int64

Popular genres on Monday morning in St. Petersburg and Moscow turned out to be similar: everywhere, as expected, pop is popular. Despite this, the ending of the top 10 for the two cities is different: in St. Petersburg, the top 10 includes jazz and Russian rap, and in Moscow th genre is 'world'.

At the end of the week, the situation does not change. Pop music still comes first. Again, the difference is noticeable only at the end of the top 10, where the 'world' genre is also present in St. Petersburg on Friday evening.

**Conclusions**

The pop genre is the undisputed leader, and the top 5 in general does not differ in both capitals. At the same time, it can be seen that the end of the list is more 'lively': for each city, more characteristic genres are distinguished, which really changes their positions depending on the day of the week and time.

# Moscow and St. Petersburg are two different capitals, two different directions in music. Is it true?

Hypothesis: St. Petersburg is rich in his rap culture, so this trend is listened to more often there, and Moscow is a city of contrasts, but the bulk of users listen to pop music.



Let's group the 'moscow_general' table by genre, count the number of comparisons of each genre using the 'count()' method, sort in descending order and save the result in the 'moscow_genres' table.

In [51]:

moscow_genres = moscow_general.groupby('genre_name')['genre_name'].count().sort_values(ascending=False)


In [52]:
moscow_genres.head(10) 

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

Let's group the 'spb_general' table by genre, count the number of compositions of each genre using the 'count()' method, sort in descending order and save the result in the 'spb_genres' table.

For compare the two cities need only first 10 rows of this table. 

In [53]:
spb_genres = spb_general.groupby('genre_name')['genre_name'].count().sort_values(ascending=False)

In [54]:
spb_genres.head(10)

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64

**Conclusions**

In Moscow, besides the absolutely popular genre of pop, there is a direction of russian popular music. This means that the interest in this genre is wider. And rap, contrary to assumption, occupies similar positions in both cities.

# Research results


Hypotheses:

* music in two cities - Moscow and St. Petersburg - is listened to in different modes;

* the lists of the ten most popular genres on Monday morning and Friday evening have characteristic differences;

* the population of the two cities prefers different musical genres.

**Overall results**

Moscow and St. Petersburg agree in tastes: popular music prevails everywhere. At the same time, there is no dependence of preferences on the day of the week in each separate city - people constantly listen to what they the like. But there are differences between the cities in terms of days of the week: Moscow listens more on Monday and Friday, while St. Petersburg, on the contrary, listens more on Wednesday, but less on Monday and Friday.


As a result, indicate the first hypothesis is confirmed, the second hypothesis is not confirmed and the third is not confirmed.